In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
question = '안녕하세요'


In [ ]:
text_prompt = f'아래 질문에 답변해주세요 : {question}'
llm.invoke(text_prompt)

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_with_template = '아래 질문에 답변해주세요\n\n {question}'
prompt_template = PromptTemplate(template=prompt_with_template, input_variables=["question"])
prompt_template.invoke({"question": question})


StringPromptValue(text='아래 질문에 답변해주세요\n\n 안녕하세요')

In [9]:
llm.invoke(prompt_template.invoke({"question": question}))

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 17, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpBD86ov7SiCNCFNGk3cnOtHCNFmj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--204b25d3-2606-4f41-843c-7861fb7673cd-0', usage_metadata={'input_tokens': 17, 'output_tokens': 10, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
prompt_chain = prompt_template | llm
prompt_chain.invoke({"question: question"})

In [10]:
%pip install -qU beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://www.inflearn.com/?srsltid=AfmBOorbUcri9mxK6CLctXQP4hJcDN_4X9TAYw0XgGXb-hZf1HtJmvxF')



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

document_list = loader.load_and_split(text_splitter)

In [13]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
    collection_name='PromptTemplateTest',
    persist_directory='./chroma'
)


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [14]:
retriever = database.as_retriever(search_kwargs={'k':1})
retriever_docs = retriever.invoke(question)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [15]:
retriever_docs

[Document(id='d7c34e63-96df-4253-8aa9-788f80cb4004', metadata={'description': '프로그래밍, 인공지능, 데이터, 마케팅, 디자인등 입문부터 실전까지 업계 최고 선배들에게 배울 수 있는 곳.', 'source': 'https://www.inflearn.com/?srsltid=AfmBOorbUcri9mxK6CLctXQP4hJcDN_4X9TAYw0XgGXb-hZf1HtJmvxF', 'title': '인프런 - 라이프타임 커리어 플랫폼', 'language': 'ko'}, page_content='인프런 - 라이프타임 커리어 플랫폼인프런 - 라이프타임 커리어 플랫폼검색검색전체전체기술 검색기술 검색AI 활용왕초보답변 활발미션모임/부트캠프난이도난이도추천순추천순')]

In [16]:
rag_prompt_template = PromptTemplate(
    template='''
    You are a helpful assistant that can answer questions about the Inflearn website.
    You are given the following context:
    {context}
    Question: {question}
''',
input_variables=["contenxt","question"]
)

rag_chain = rag_prompt_template | llm
rag_chain.invoke({"context": retriever_docs, "question": question})

AIMessage(content='안녕하세요! 어떻게 도와드릴까요? 인프런에 대한 질문이 있으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 253, 'total_tokens': 275, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpBps22VwoV0z6N8nQYrikbBoz4gu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b3dae76d-18d8-421e-bbed-d206966e0930-0', usage_metadata={'input_tokens': 253, 'output_tokens': 22, 'total_tokens': 275, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Users\1\miniconda3\envs\disRagEnv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
rag_chain = prompt | llm
rag_chain.invoke({"context": retriever_docs, "question": question})

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 277, 'total_tokens': 287, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpBvTTRaqltYt77dEZ6A1BKf3I8Kt', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e0af5e23-7659-48a2-b230-59b65834f72a-0', usage_metadata={'input_tokens': 277, 'output_tokens': 10, 'total_tokens': 287, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})